In [1]:
import sys
import os
# Add the src directory to Python path so model.py can find ssn and net modules
sys.path.append(os.path.abspath('../src'))

In [2]:
import numpy as np
from loguru import logger
# load the data
path = '../data_result/raw_data/gauss_cos_31x31.npy'
data = np.load(path)
logger.info(f"Loaded data with shape: {data.shape}, dtype: {data.dtype}")

2025-09-23 13:05:10.301 | INFO     | __main__:<module>:6 - Loaded data with shape: (961,), dtype: [('x', '<f8', (2,)), ('dv', '<f8', (2,)), ('v', '<f8')]


In [3]:
# Initialize the parameter
power = 2.1
M = 50 # number greedy insertion selected
num_iterations = 10
loss_weights = (1.0, 0.0)
pruning_threshold = 1e-15

gamma = 5.0
alpha = 1e-5
lr_adam = 1e-5
regularization = (gamma, alpha) 
th = 0.0

In [4]:
from src.model import model
import torch
# Initialize the models
model_1 = model(
    activation=torch.relu, 
    power=power, 
    regularization=regularization, 
    optimizer='Adam', 
    loss_weights=loss_weights, 
    th = th
    )
model_2 = model(
    activation=torch.relu, 
    power=power, 
    regularization=regularization, 
    optimizer='SSN_TR', 
    loss_weights=loss_weights, 
    th=th, 
    train_outerweights=True)

2025-09-23 13:05:11 | INFO     | src.model:_configure_logger:105 - Model initialized
2025-09-23 13:05:11 | INFO     | src.model:_configure_logger:105 - Model initialized


In [5]:
# prepare the data
data_train, data_valid = model_1._prepare_data(data)


2025-09-23 13:05:11 | INFO     | src.model:_prepare_data:150 - Training set: 864 samples, Validation set: 97 samples
2025-09-23 13:05:11 | INFO     | src.model:_prepare_data:153 - Data ranges - x: [-1.00, 1.00], v: [-0.72, 1.00], dv: [-5.86, 5.86]


In [6]:
from src.PDAP import retrain

best_iteration, history = retrain(
    data_train, 
    data_valid, 
    model_1, 
    model_2, 
    num_iterations, 
    M, 
    pruning_threshold
    )

2025-09-23 13:05:11 | INFO     | src.PDAP:retrain:56 - Iteration 0 - Starting...
2025-09-23 13:05:11 | INFO     | src.model:train:307 - Starting network training session
2025-09-23 13:05:11 | INFO     | src.model:_create_network:194 - Creating network with 50 neurons
2025-09-23 13:05:12 | INFO     | src.model:_setup_optimizer:244 - Using Adam optimizer with lr=0.01
2025-09-23 13:05:12 | INFO     | src.model:train:322 - Training hyperparameters: iterations=1000, batch_size=1620, display_every=100
2025-09-23 13:05:12 | INFO     | src.model:train:323 - Loss weights: value=1.0, gradient=0.0
2025-09-23 13:05:12 | INFO     | src.model:train:367 - Epoch 0: Train Loss = 2.698220, Val Loss = 0.348824
2025-09-23 13:05:12 | INFO     | src.model:train:367 - Epoch 100: Train Loss = 0.230126, Val Loss = 0.251590
2025-09-23 13:05:12 | INFO     | src.model:train:367 - Epoch 200: Train Loss = 0.202792, Val Loss = 0.278574
2025-09-23 13:05:12 | INFO     | src.model:train:367 - Epoch 300: Train Loss = 0.

## L1 ##

In [7]:
# Initialize the parameter
power = 2.1
M = 50 # number greedy insertion selected
num_iterations = 10
loss_weights = (1.0, 0.0)
pruning_threshold = 1e-13

gamma = 1e-10
alpha = 1e-5
regularization = (gamma, alpha) 
th = 1.0

In [8]:
model_1 = model(
    activation=torch.relu, 
    power=power, 
    regularization=regularization, 
    optimizer='Adam', 
    loss_weights=loss_weights, 
    th = th
    )
model_2 = model(
    activation=torch.relu, 
    power=power, 
    regularization=regularization, 
    optimizer='SSN_TR', 
    loss_weights=loss_weights, 
    th=th, 
    train_outerweights=True)

2025-09-23 13:07:38 | INFO     | src.model:_configure_logger:105 - Model initialized
2025-09-23 13:07:38 | INFO     | src.model:_configure_logger:105 - Model initialized


In [9]:
best_iteration_L1, history_L1 = retrain(
    data_train, 
    data_valid, 
    model_1, 
    model_2, 
    num_iterations, 
    M, 
    pruning_threshold
    )

2025-09-23 13:07:38 | INFO     | src.PDAP:retrain:56 - Iteration 0 - Starting...
2025-09-23 13:07:38 | INFO     | src.model:train:307 - Starting network training session
2025-09-23 13:07:38 | INFO     | src.model:_create_network:194 - Creating network with 50 neurons
2025-09-23 13:07:38 | INFO     | src.model:_setup_optimizer:244 - Using Adam optimizer with lr=0.01
2025-09-23 13:07:38 | INFO     | src.model:train:322 - Training hyperparameters: iterations=1000, batch_size=1620, display_every=100
2025-09-23 13:07:38 | INFO     | src.model:train:323 - Loss weights: value=1.0, gradient=0.0
2025-09-23 13:07:38 | INFO     | src.model:train:367 - Epoch 0: Train Loss = 0.984487, Val Loss = 0.739228
2025-09-23 13:07:39 | INFO     | src.model:train:367 - Epoch 100: Train Loss = 0.210523, Val Loss = 0.312204
2025-09-23 13:07:39 | INFO     | src.model:train:367 - Epoch 200: Train Loss = 0.173318, Val Loss = 0.358600
2025-09-23 13:07:39 | INFO     | src.model:train:367 - Epoch 300: Train Loss = 0.

In [10]:
# def save(self):
#     """Save training history."""
#     timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
#     filename = f"training_history_{timestamp}.pkl"
#     filepath = os.path.join(self.stats_dir, filename)
    
#     with open(filepath, 'wb') as f:
#         pickle.dump(self.history, f)
        
#     logger.info(f"Saved training history to {filepath}")
#     return filepath